In [ ]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, LearningRateScheduler
import time

PLOT=True
# PLOT=False

SEQ_LEN=60

In [ ]:
%load_ext tensorboard

In [ ]:
url = "./datasets/DAT_ASCII_EURUSD_M1_2017.csv"
df = pd.read_csv(url, names=list(["date","open", "high", "low", "close", "volume"]), header=None, sep=";")
df.drop("volume", 1, inplace=True)
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df.set_index('date', inplace=True)

df.head()

In [ ]:
if PLOT:
    df.plot(subplots=True, layout=(2, 2), figsize=(40, 20), sharex=False)

In [ ]:
data_set = df.iloc[:, 2:3].values #close
print(data_set)

In [ ]:
print(data_set.shape)

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(data_set)

In [ ]:
X_train = []
y_train = []
for i in range(SEQ_LEN, training_set_scaled.size-20):
    X_train.append(training_set_scaled[i-SEQ_LEN:i, 0])
    y_train.append(training_set_scaled[i+1:i+6, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train[0])
print(y_train[0])

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train[0])

In [ ]:
print(y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [12]:
model = Sequential()

model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=5))

opt = Adam(lr=0.001, decay=1e-6)

NAME = f"PRED-{int(time.time())}"  
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME, update_freq=25))
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.0001)


def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    return lr / (epoch*3)  

lr_scheduler_cb=LearningRateScheduler(schedule=scheduler)

model.compile(optimizer=opt,loss='mean_squared_error')

model.fit(X_train,y_train,epochs=5,batch_size=256,validation_data=(X_test, y_test),callbacks=[lr_scheduler_cb])

model.save(f"models/{NAME}_model.h5")

score=model.evaluate(X_test, y_test, verbose=1) 

Epoch 1/30
 161/1162 [===>..........................] - ETA: 2:17 - loss: 0.0280

KeyboardInterrupt: 

In [ ]:
print(NAME)